In [47]:
import pandas as pd

In [1]:
import os
import json
import traceback

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
key= os.getenv("OPENAI_KEY")

In [4]:
from dotenv import load_dotenv

In [5]:
llm= ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.3)

In [11]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001ADF68E5EE0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001ADF68F08B0>, root_client=<openai.OpenAI object at 0x000001ADF689E970>, root_async_client=<openai.AsyncOpenAI object at 0x000001ADF68E5F10>, temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='')

In [6]:
#from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
Template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs. 
### RESPONSE_JSON
{response_json}"""

In [8]:
Template2="""You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [9]:
response_json={
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
quiz_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=Template
)

In [11]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)

C:\Users\saich\AppData\Local\Temp\ipykernel_7236\2504320390.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)


In [12]:
quiz_evaluation=PromptTemplate(
    input_variables=["subject","quiz"],
    template=Template2
)

In [13]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation,output_key="review",verbose=True)

In [14]:
generate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=True)

In [15]:
json.dumps(response_json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [19]:
with open("D:\GenAI\Projects\mcqgen\mcqtext.txt", "r",encoding="utf-8") as f:
    text=f.read()

In [20]:
TEXT=text
NUMBER=3
SUBJECT="ARTIFICIAL INTELLIGENCE"
TONE="MEDIUM"

In [21]:
with get_openai_callback() as cb:
    response=generate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(response_json)
        }
        )

C:\Users\saich\AppData\Local\Temp\ipykernel_7236\2763546850.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:One of the pioneers of intelligent computers was Alan Turing. Often credited with laying the foundational theory for modern computing, he proposed the concept of a machine that could simulate any other machine (the Turing Machine) in the 1930s. His famous Turing Test, proposed in the 1950s, was a fundamental contribution to the field of AI, challenging whether machines could mimic human intelligence. However, it wasn’t until 1955 that John McCarthy founded AI as an academic discipline and called it “artificial intelligence.” He also developed a programming language called LISP in the late 1950s, which became crucial to AI development.

Deep learning started to develop in the 1990s as researchers began creating early versions of deep learning algorithms, which were initially limited by the need for greater computational power. Then, in 1997, AI won during a chess game. This victor

In [22]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1380
Prompt Tokens:889
Completion Tokens:491
Total Cost:0.0011810000000000002


In [23]:
quiz=response.get("quiz")

In [24]:
quiz

'{\n    "1": {\n        "mcq": "Who is often credited with laying the foundational theory for modern computing?",\n        "options": {\n            "a": "John McCarthy",\n            "b": "Alan Turing",\n            "c": "Garry Kasparov",\n            "d": "Deep Blue"\n        },\n        "correct": "b"\n    },\n    "2": {\n        "mcq": "Which programming language was developed by John McCarthy in the late 1950s and became crucial to AI development?",\n        "options": {\n            "a": "Python",\n            "b": "Java",\n            "c": "LISP",\n            "d": "C++"\n        },\n        "correct": "c"\n    },\n    "3": {\n        "mcq": "In which decade did deep learning start to develop?",\n        "options": {\n            "a": "1960s",\n            "b": "1980s",\n            "c": "1990s",\n            "d": "2000s"\n        },\n        "correct": "c"\n    }\n}'

In [25]:
review=response.get("review")

In [56]:
review

'The quiz is moderately complex for ARTIFICIAL INTELLIGENCE students. The questions cover key historical events and figures in AI. However, the language and tone could be simplified for better understanding. For example, "Who proposed the concept of a machine that could simulate any other machine in the 1930s?" could be rephrased as "Who first suggested a machine that could mimic other machines in the 1930s?" to make it more student-friendly.'

In [26]:
formatted_quiz = json.dumps(json.loads(quiz), indent=None, separators=(',', ':'))

In [27]:
formatted_quiz

'{"1":{"mcq":"Who is often credited with laying the foundational theory for modern computing?","options":{"a":"John McCarthy","b":"Alan Turing","c":"Garry Kasparov","d":"Deep Blue"},"correct":"b"},"2":{"mcq":"Which programming language was developed by John McCarthy in the late 1950s and became crucial to AI development?","options":{"a":"Python","b":"Java","c":"LISP","d":"C++"},"correct":"c"},"3":{"mcq":"In which decade did deep learning start to develop?","options":{"a":"1960s","b":"1980s","c":"1990s","d":"2000s"},"correct":"c"}}'

In [35]:
quiz=json.loads(formatted_quiz)

In [36]:
quiz

{'1': {'mcq': 'Who is often credited with laying the foundational theory for modern computing?',
  'options': {'a': 'John McCarthy',
   'b': 'Alan Turing',
   'c': 'Garry Kasparov',
   'd': 'Deep Blue'},
  'correct': 'b'},
 '2': {'mcq': 'Which programming language was developed by John McCarthy in the late 1950s and became crucial to AI development?',
  'options': {'a': 'Python', 'b': 'Java', 'c': 'LISP', 'd': 'C++'},
  'correct': 'c'},
 '3': {'mcq': 'In which decade did deep learning start to develop?',
  'options': {'a': '1960s', 'b': '1980s', 'c': '1990s', 'd': '2000s'},
  'correct': 'c'}}

In [38]:
for key,value in quiz.items():
    print(key,value)

1 {'mcq': 'Who is often credited with laying the foundational theory for modern computing?', 'options': {'a': 'John McCarthy', 'b': 'Alan Turing', 'c': 'Garry Kasparov', 'd': 'Deep Blue'}, 'correct': 'b'}
2 {'mcq': 'Which programming language was developed by John McCarthy in the late 1950s and became crucial to AI development?', 'options': {'a': 'Python', 'b': 'Java', 'c': 'LISP', 'd': 'C++'}, 'correct': 'c'}
3 {'mcq': 'In which decade did deep learning start to develop?', 'options': {'a': '1960s', 'b': '1980s', 'c': '1990s', 'd': '2000s'}, 'correct': 'c'}


In [44]:
quiz_table_data=[]
for _,value in quiz.items():
    quiz_table_data.append([value["mcq"],"|".join(f"{opt} | {opt_val}" for opt,opt_val in value["options"].items()),value["correct"]])

In [45]:
quiz_table_data

[['Who is often credited with laying the foundational theory for modern computing?',
  'a | John McCarthy|b | Alan Turing|c | Garry Kasparov|d | Deep Blue',
  'b'],
 ['Which programming language was developed by John McCarthy in the late 1950s and became crucial to AI development?',
  'a | Python|b | Java|c | LISP|d | C++',
  'c'],
 ['In which decade did deep learning start to develop?',
  'a | 1960s|b | 1980s|c | 1990s|d | 2000s',
  'c']]

In [49]:
quiz=pd.DataFrame(quiz_table_data)
quiz=quiz.to_csv("Quiz_pdf.csv",index=False)